In [ ]:
from urllib.request import urlretrieve
urlretrieve('https://archive.ics.uci.edu/static/public/53/iris.zip', 'data.zip')

In [ ]:
!mkdir data
!unzip data.zip -d data/

In [ ]:
# import pandas as pd
#
# # download iris data and read it into a dataframe
# url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
# df = pd.read_csv(url, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
# df.describe()

In [ ]:
import pandas as pd
df =  pd.read_csv('data/iris.data', header=None)
df.head()

In [ ]:
df[4] = df[4].replace('Iris-setosa', 0)
df[4] = df[4].replace('Iris-virginica', 1)
df[4] = df[4].replace('Iris-versicolor', 2)
df = df[[4,0,1,2,3]]

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
training_data = df[:120]
testing_data = df[120:]

In [ ]:
import boto3
bucket_name = 'sagemaker-buildml'

In [ ]:
training_data.to_csv('data.csv', header=False, index=False)
key = 'data/train/data'
# url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

In [ ]:
key = 'data/val/data'
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

In [ ]:
# ()
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

key = 'model/xgb_model'
s3_output_location = 's3://{}/{}'.format(bucket_name, key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.large',
    train_volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

xgb_model.set_hyperparameters(max_depth=5,
                             eta=0.2,
                             gamma=4,
                             min_child_weight=6,
                             silent=0,
                             objective='multi:softmax',
                             num_class=3,
                             num_round=10
                            )

In [ ]:
# ()
training_data=  's3://{}/{}'.format(bucket_name, 'data/train')
val_data=  's3://{}/{}'.format(bucket_name, 'data/val')

train_channel = sagemaker.session.s3_input(training_data, content_type='text/csv')
val_channel = sagemaker.session.s3_input(val_data, content_type='text/csv')

data_channels = {'train': train_channel, 'validation': val_channel}

xgb_model.fit(inputs=data_channels)

In [ ]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                 instance_type='ml.m5.large'
                                )